In [1]:
# takes 60s per 40 api calls.
# download player tournament data. skip if already downloaded.

# previous steps:
# none

# next steps:
# create sql db of player tournament data.

# todo:
# 

In [2]:
import pandas as pd
import pathlib
from collections import defaultdict
import mlBridgeLib

In [3]:
# override pandas display options
mlBridgeLib.pd_options_display()

In [4]:
rootPath = pathlib.Path('e:/bridge/data')
acblPath = rootPath.joinpath('acbl')
acblPath.mkdir(parents=True,exist_ok=True)

In [5]:
bprPath = rootPath.joinpath('bpr')
allmps_file = bprPath.joinpath('allmps.pkl')
df = pd.read_pickle(allmps_file)
df

,Rank,MP's,Unit,Name,player_id
0,1,92862.58,128,"Meckstroth, Jeff",[4580699]
1,2,87910.58,176,"Passell, Mike",[1622382]
2,3,78168.46,128,"Rodwell, Eric",[5482658]
3,4,76525.13,174,"Wold, Eddie",[3443949]
4,5,71817.34,210,"Lair, Mark",[5830400]
5,6,62350.16,538,"Itabashi, Mark",[6811434]
6,7,51506.72,373,"Hampson, Geoff",[9984488]
7,8,50419.44,176,"Compton, Chris",[8007713]
8,9,47550.39,373,"Levin, Robert",[7279515]
9,10,46419.44,373,"Cheek, Curtis",[2931060]


In [6]:
bprPath = rootPath.joinpath('bpr')
allmps_file = bprPath.joinpath('allmps.pkl')
df = pd.read_pickle(allmps_file)
df

,Rank,MP's,Unit,Name,player_id
0,1,92862.58,128,"Meckstroth, Jeff",[4580699]
1,2,87910.58,176,"Passell, Mike",[1622382]
2,3,78168.46,128,"Rodwell, Eric",[5482658]
3,4,76525.13,174,"Wold, Eddie",[3443949]
4,5,71817.34,210,"Lair, Mark",[5830400]
5,6,62350.16,538,"Itabashi, Mark",[6811434]
6,7,51506.72,373,"Hampson, Geoff",[9984488]
7,8,50419.44,176,"Compton, Chris",[8007713]
8,9,47550.39,373,"Levin, Robert",[7279515]
9,10,46419.44,373,"Cheek, Curtis",[2931060]


In [7]:
# read file containing acbl api bearer secret.
bearer_file = pathlib.Path('c:/sw/bridge/BridgePowerRatings/acbl_api_authorization_bearer_secret.txt')
with open(bearer_file,'rb') as f:
    bearer = f.read()

In [ ]:
# takes 60s per 40 api calls.
# call acbl api to retrieve player's tournament data.
# todo: inexplicable variance between 'total' in json and sessions_count. Why? varaince is 0 to -15-ish.
import requests
import urllib
import time
import json
headers = {'accept':'application/json', 'Authorization':bearer[len('Authorization: '):]}
start_time = time.time()
get_count = 0
for n,i in enumerate(df['Rank'].astype('float32').sort_values().index): # all this work needed to sort by rank
    r = df.iloc[i]
    player_ids = r['player_id']
    assert type(player_ids) is list, [type(player_ids), player_ids]
    print(f"{time.strftime('%X')} player_id list:{player_ids} Name:{r['Name']} Rank:{r['Rank']}")
    for player_id in player_ids:
        if player_id.startswith('tmp:'): # todo: looks like this didn't work? Needs testing.
            print('Skipping:',player_id)
            continue
        else:
            print('player_id:',player_id)
        dirPath = acblPath.joinpath('players/'+player_id+'/tournaments')
        if dirPath.exists():
            session_file_count = len(list(dirPath.glob('*.session.json')))
            print(f'dir exists: file count:{session_file_count} dir:{dirPath}')
            if session_file_count > 0: # todo: temp?
                print(f'dir not empty -- skipping')
                continue
        else:
            print(f'creating dir:{dirPath}')
            dirPath.mkdir(parents=True,exist_ok=True)
            session_file_count = 0
        path = 'https://api.acbl.org/v1/tournament/player/history_query'
        query = {'acbl_number':player_id,'page':1,'page_size':50,'start_date':'1900-01-01'}
        params = urllib.parse.urlencode(query)
        url = path+'?'+params
        sessions_count = 1 # must start at 1 to match sessions_total
        while url:
            get_count += 1
            print(f"{n}/{len(df)} gets:{get_count} rate:{round((time.time()-start_time)/get_count,2)} url:{url}")
            #time.sleep(1) # throttle api calling. Maybe not needed as api is taking longer than 1s.
            response = requests.get(url, headers=headers)
            if response.status_code == 500: # unknown response code. try skipping player
                print(f'500: count:{get_count} skipping') # 4476921 - Thx Merle.
                next_page_url = None
                break
            assert response.status_code == 200, (n, url, response.status_code)
            json_response = response.json()
            json_pretty = json.dumps(json_response, indent=4)
            #print(json_pretty)
            next_page_url = json_response['next_page_url']
            sessions_total = json_response['total'] # is same for every page
            if sessions_total == session_file_count: # sometimes won't agree because identical sessions. older scores?
                print(f'File count correct: {dirPath}: terminating {player_id} early.')
                next_page_url = None
                sessions_count = sessions_total
                break
            for data in json_response['data']:
                sessions_count += 1 # todo: oops, starts first one at 2. need to move
                session_id = data['session_id']
                filePath = dirPath.joinpath(session_id+'.session.json')
                if filePath.exists():
                    print(f'{sessions_count}/{sessions_total}: File exists: {filePath}: skipping')
                    continue
                print(f'{sessions_count}/{sessions_total}: Writing:',filePath)
                with open(filePath,'w',encoding='UTF8') as f:
                    f.write(json.dumps(data, indent=4))
            url = next_page_url
        if sessions_count != sessions_total:
            print(f'Session count mismatch: {dirPath}: variance:{sessions_count-sessions_total}')

18:09:18 player_id list:['4580699'] Name:Meckstroth, Jeff Rank:1
player_id: 4580699
dir exists: file count:674 dir:e:\bridge\data\acbl\players\4580699\tournaments
dir not empty -- skipping
18:09:18 player_id list:['1622382'] Name:Passell, Mike Rank:2
player_id: 1622382
dir exists: file count:724 dir:e:\bridge\data\acbl\players\1622382\tournaments
dir not empty -- skipping
18:09:18 player_id list:['5482658'] Name:Rodwell, Eric Rank:3
player_id: 5482658
dir exists: file count:660 dir:e:\bridge\data\acbl\players\5482658\tournaments
dir not empty -- skipping
18:09:18 player_id list:['3443949'] Name:Wold, Eddie Rank:4
player_id: 3443949
dir exists: file count:787 dir:e:\bridge\data\acbl\players\3443949\tournaments
dir not empty -- skipping
18:09:18 player_id list:['5830400'] Name:Lair, Mark Rank:5
player_id: 5830400
dir exists: file count:732 dir:e:\bridge\data\acbl\players\5830400\tournaments
dir not empty -- skipping
18:09:18 player_id list:['6811434'] Name:Itabashi, Mark Rank:6
player_id

File count correct: e:\bridge\data\acbl\players\3530000\tournaments: terminating 3530000 early.
18:09:23 player_id list:['8914737'] Name:Stansby, Lew Rank:35
player_id: 8914737
dir exists: file count:285 dir:e:\bridge\data\acbl\players\8914737\tournaments
dir not empty -- skipping
18:09:23 player_id list:['9593322'] Name:Onstott, John Rank:36
player_id: 9593322
dir exists: file count:495 dir:e:\bridge\data\acbl\players\9593322\tournaments
dir not empty -- skipping
18:09:23 player_id list:['1182730'] Name:DeMartino, Richard Rank:37
player_id: 1182730
dir exists: file count:646 dir:e:\bridge\data\acbl\players\1182730\tournaments
dir not empty -- skipping
18:09:23 player_id list:['5573122'] Name:Morse, Dan Rank:38
player_id: 5573122
dir exists: file count:469 dir:e:\bridge\data\acbl\players\5573122\tournaments
dir not empty -- skipping
18:09:23 player_id list:['4761685'] Name:Mohan, John Rank:39
player_id: 4761685
dir exists: file count:737 dir:e:\bridge\data\acbl\players\4761685\tourname

File count correct: e:\bridge\data\acbl\players\9503943\tournaments: terminating 9503943 early.
player_id: 6133126
dir exists: file count:445 dir:e:\bridge\data\acbl\players\6133126\tournaments
dir not empty -- skipping
player_id: 8983933
dir exists: file count:4 dir:e:\bridge\data\acbl\players\8983933\tournaments
dir not empty -- skipping
player_id: 6039758
dir exists: file count:132 dir:e:\bridge\data\acbl\players\6039758\tournaments
dir not empty -- skipping
Skipping: tmp:2e16a3aa-29a3-4773-ad8a-66c686ff4cb7
Skipping: tmp:ff92aaf1-f746-4f5f-8c34-496a6bb6e90e
18:09:26 player_id list:['8149887'] Name:Wolpert, Gavin Rank:90
player_id: 8149887
dir exists: file count:367 dir:e:\bridge\data\acbl\players\8149887\tournaments
dir not empty -- skipping
18:09:26 player_id list:['4057163', '5512883', '8888639'] Name:Morris, Robert Rank:91
player_id: 4057163
dir exists: file count:953 dir:e:\bridge\data\acbl\players\4057163\tournaments
dir not empty -- skipping
player_id: 5512883
dir exists: fil

dir exists: file count:10 dir:e:\bridge\data\acbl\players\3194213\tournaments
dir not empty -- skipping
18:09:29 player_id list:['3966968'] Name:Clough, Rebecca Rank:167
player_id: 3966968
dir exists: file count:590 dir:e:\bridge\data\acbl\players\3966968\tournaments
dir not empty -- skipping
18:09:29 player_id list:['4705327'] Name:Gardner, Robert Rank:168
player_id: 4705327
dir exists: file count:842 dir:e:\bridge\data\acbl\players\4705327\tournaments
dir not empty -- skipping
18:09:29 player_id list:['5653576'] Name:Crawford, Muh-Sha Rank:169
player_id: 5653576
dir exists: file count:470 dir:e:\bridge\data\acbl\players\5653576\tournaments
dir not empty -- skipping
18:09:29 player_id list:['7283407'] Name:Harris, Joe Rank:170
player_id: 7283407
dir exists: file count:430 dir:e:\bridge\data\acbl\players\7283407\tournaments
dir not empty -- skipping
18:09:29 player_id list:['4333713'] Name:Shoup, Russell Rank:171
player_id: 4333713
dir exists: file count:248 dir:e:\bridge\data\acbl\pla

File count correct: e:\bridge\data\acbl\players\8326991\tournaments: terminating 8326991 early.
Skipping: tmp:6a13a332-725e-4051-9ae5-d69acedcea40
Skipping: tmp:15209ec3-a334-4add-a957-9c545cf6de8b
Skipping: tmp:5ca896ee-dfa3-4732-b086-f75597251642
Skipping: tmp:1b4ef8c9-a365-4baa-9d16-62dc92b72501
Skipping: tmp:e15ab6fd-4596-4830-a812-06015c77c7f4
Skipping: tmp:afdb3969-1e53-44ac-90ac-0d8c87f0dbac
18:09:30 player_id list:['4720024'] Name:Cayne, James Rank:212
player_id: 4720024
dir exists: file count:139 dir:e:\bridge\data\acbl\players\4720024\tournaments
dir not empty -- skipping
18:09:30 player_id list:['4393252'] Name:Levine, Mike Rank:213
player_id: 4393252
dir exists: file count:304 dir:e:\bridge\data\acbl\players\4393252\tournaments
dir not empty -- skipping
18:09:30 player_id list:['6839959'] Name:Cooper, Kitty Rank:214
player_id: 6839959
dir exists: file count:332 dir:e:\bridge\data\acbl\players\6839959\tournaments
dir not empty -- skipping
18:09:30 player_id list:['6172067'] 

...

252/254: Writing: e:\bridge\data\acbl\players\6307620\tournaments\NABC133-0705-1.session.json
253/254: Writing: e:\bridge\data\acbl\players\6307620\tournaments\NABC133-0634-2.session.json
254/254: Writing: e:\bridge\data\acbl\players\6307620\tournaments\NABC133-0503-2.session.json
255/254: Writing: e:\bridge\data\acbl\players\6307620\tournaments\NABC133-0503-1.session.json
Session count mismatch: e:\bridge\data\acbl\players\6307620\tournaments: variance:1
18:17:21 player_id list:['5206030'] Name:Deutsch, Barbara Rank:3337
player_id: 5206030
dir exists: file count:0 dir:e:\bridge\data\acbl\players\5206030\tournaments
3336/5000 gets:389 rate:1.24 url:https://api.acbl.org/v1/tournament/player/history_query?acbl_number=5206030&page=1&page_size=50&start_date=1900-01-01
2/16: Writing: e:\bridge\data\acbl\players\5206030\tournaments\1908027-SS03-5.session.json
3/16: Writing: e:\bridge\data\acbl\players\5206030\tournaments\1908027-SS02-4.session.json
4/16: Writing: e:\bridge\data\acbl\players\

102/125: Writing: e:\bridge\data\acbl\players\6057926\tournaments\1603024-0411-1.session.json
103/125: Writing: e:\bridge\data\acbl\players\6057926\tournaments\1603024-0401-1.session.json
104/125: Writing: e:\bridge\data\acbl\players\6057926\tournaments\1510100-2501-2.session.json
105/125: Writing: e:\bridge\data\acbl\players\6057926\tournaments\1510100-2401-1.session.json
106/125: Writing: e:\bridge\data\acbl\players\6057926\tournaments\1508041-9101-2.session.json
107/125: Writing: e:\bridge\data\acbl\players\6057926\tournaments\1508041-3101-1.session.json
108/125: Writing: e:\bridge\data\acbl\players\6057926\tournaments\1508041-3111-1.session.json
109/125: Writing: e:\bridge\data\acbl\players\6057926\tournaments\1505163-2411-2.session.json
110/125: Writing: e:\bridge\data\acbl\players\6057926\tournaments\1504006-0412-2.session.json
111/125: Writing: e:\bridge\data\acbl\players\6057926\tournaments\1504006-0301-1.session.json
112/125: Writing: e:\bridge\data\acbl\players\6057926\tourna

102/164: Writing: e:\bridge\data\acbl\players\2409232\tournaments\1708002-09PA-2.session.json
103/164: Writing: e:\bridge\data\acbl\players\2409232\tournaments\1708002-SS01-2.session.json
104/164: Writing: e:\bridge\data\acbl\players\2409232\tournaments\1708002-SS03-1.session.json
105/164: Writing: e:\bridge\data\acbl\players\2409232\tournaments\1708002-071A-2.session.json
106/164: Writing: e:\bridge\data\acbl\players\2409232\tournaments\1707082-2301-2.session.json
107/164: Writing: e:\bridge\data\acbl\players\2409232\tournaments\1703034-26TO-2.session.json
108/164: Writing: e:\bridge\data\acbl\players\2409232\tournaments\1703034-252P-1.session.json
109/164: Writing: e:\bridge\data\acbl\players\2409232\tournaments\1703034-252T-1.session.json
110/164: Writing: e:\bridge\data\acbl\players\2409232\tournaments\1703034-251P-1.session.json
111/164: Writing: e:\bridge\data\acbl\players\2409232\tournaments\1703034-242P-1.session.json
112/164: Writing: e:\bridge\data\acbl\players\2409232\tourna

52/94: Writing: e:\bridge\data\acbl\players\1717952\tournaments\1807096-3001-2.session.json
53/94: Writing: e:\bridge\data\acbl\players\1717952\tournaments\1807096-3001-1.session.json
54/94: Writing: e:\bridge\data\acbl\players\1717952\tournaments\1804023-2701-2.session.json
55/94: Writing: e:\bridge\data\acbl\players\1717952\tournaments\1804023-2701-1.session.json
56/94: Writing: e:\bridge\data\acbl\players\1717952\tournaments\1803128-0401-2.session.json
57/94: Writing: e:\bridge\data\acbl\players\1717952\tournaments\1803128-0301-2.session.json
58/94: Writing: e:\bridge\data\acbl\players\1717952\tournaments\1803128-0301-1.session.json
59/94: Writing: e:\bridge\data\acbl\players\1717952\tournaments\1801033-SBAP-2.session.json
60/94: Writing: e:\bridge\data\acbl\players\1717952\tournaments\1801033-SBAP-1.session.json
61/94: Writing: e:\bridge\data\acbl\players\1717952\tournaments\1709130-MBAS-2.session.json
62/94: Writing: e:\bridge\data\acbl\players\1717952\tournaments\1709130-SBAP-2.s

52/95: Writing: e:\bridge\data\acbl\players\1147056\tournaments\1703028-2501-2.session.json
53/95: Writing: e:\bridge\data\acbl\players\1147056\tournaments\1703028-2501-1.session.json
54/95: Writing: e:\bridge\data\acbl\players\1147056\tournaments\1701032-2801-2.session.json
55/95: Writing: e:\bridge\data\acbl\players\1147056\tournaments\1701032-2801-1.session.json
56/95: Writing: e:\bridge\data\acbl\players\1147056\tournaments\1609038-251A-2.session.json
57/95: Writing: e:\bridge\data\acbl\players\1147056\tournaments\1609038-2401-2.session.json
58/95: Writing: e:\bridge\data\acbl\players\1147056\tournaments\1609038-2401-1.session.json
59/95: Writing: e:\bridge\data\acbl\players\1147056\tournaments\1609001-0301-2.session.json
60/95: Writing: e:\bridge\data\acbl\players\1147056\tournaments\1609001-0301-1.session.json
61/95: Writing: e:\bridge\data\acbl\players\1147056\tournaments\1609001-0201-2.session.json
62/95: Writing: e:\bridge\data\acbl\players\1147056\tournaments\1609001-0201-1.s